In [7]:
from dotenv import load_dotenv

load_dotenv()
from graphrag_sdk import (
    Ontology,
    Entity,
    Relation,
    Attribute,
    AttributeType,
    KnowledgeGraph,
    KnowledgeGraphModelConfig,
)
from graphrag_sdk.orchestrator import Orchestrator
from graphrag_sdk.agents.kg_agent import KGAgent
from graphrag_sdk.models.openai import OpenAiGenerativeModel
from json import loads
import os
import vertexai


# Initialize the Vertex AI client
vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("REGION"))


In [8]:
def import_data():
        with open("data/cities.json") as f:
            cities = loads(f.read())
        with open("data/restaurants.json") as f:
            restaurants = loads(f.read())
        with open("data/attractions.json") as f:
            attractions = loads(f.read())

        for city in cities:
            restaurants_kg.add_node(
                "City",
                {
                    "name": city["name"],
                    "weather": city["weather"],
                    "population": city["population"],
                },
            )
            restaurants_kg.add_node("Country", {"name": city["country"]})
            restaurants_kg.add_edge(
                "IN_COUNTRY",
                "City",
                "Country",
                {"name": city["name"]},
                {"name": city["country"]},
            )

            attractions_kg.add_node(
                "City",
                {
                    "name": city["name"],
                    "weather": city["weather"],
                    "population": city["population"],
                },
            )
            attractions_kg.add_node("Country", {"name": city["country"]})
            attractions_kg.add_edge(
                "IN_COUNTRY",
                "City",
                "Country",
                {"name": city["name"]},
                {"name": city["country"]},
            )

        for restaurant in restaurants:
            restaurants_kg.add_node(
                "Restaurant",
                {
                    "name": restaurant["name"],
                    "description": restaurant["description"],
                    "rating": restaurant["rating"],
                    "food_type": restaurant["food_type"],
                },
            )
            restaurants_kg.add_edge(
                "IN_CITY",
                "Restaurant",
                "City",
                {"name": restaurant["name"]},
                {"name": restaurant["city"]},
            )

        for attraction in attractions:
            attractions_kg.add_node(
                "Attraction",
                {
                    "name": attraction["name"],
                    "description": attraction["description"],
                    "type": attraction["type"],
                },
            )
            attractions_kg.add_edge(
                "IN_CITY",
                "Attraction",
                "City",
                {"name": attraction["name"]},
                {"name": attraction["city"]},
            )

In [9]:
restaurants_ontology = Ontology()
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)

attractions_ontology = Ontology()
attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)

model = OpenAiGenerativeModel("gpt-4o")
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

import_data()

restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="""
I'm a restaurant agent, specialized in finding the best restaurants for you. 
""",
)

attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="""
I'm an attractions agent, specialized in finding the best attractions for you. 
""",
)

orchestrator = Orchestrator(
    model,
    backstory="You are a trip planner, and you want to provide the best possible itinerary for your clients.",
)

orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

In [12]:
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="""
I'm a restaurant agent, specialized in finding the best restaurants for you. 
""",
)

attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="""
I'm an attractions agent, specialized in finding the best attractions for you. 
""",
)

orchestrator = Orchestrator(model)

orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

runner = orchestrator.ask("Write me a 3 day itinerary for a trip to Italy. Do not ask any questions to me, just provide your best itinerary.")

In [13]:
print(runner.output)

Thank you for your patience. Here is a detailed 3-day itinerary for your trip to Italy, capturing both top dining locations and must-visit attractions:

### Day 1: Rome
**Morning:**
1. **Colosseum and Roman Forum** - Start your day by exploring these iconic historical landmarks that provide a glimpse into ancient Roman life.

**Lunch:**
2. **Roscioli** - A renowned restaurant and deli where you can enjoy classic Italian dishes with fresh ingredients.

**Afternoon:**
3. **Vatican Museums and St. Peter’s Basilica** - Tour through one of the largest museums in the world and marvel at the grandeur of St. Peter’s Basilica.

**Dinner:**
4. **Antico Arco** - Located on Gianicolo Hill, this restaurant offers a mixture of traditional and contemporary Italian cuisine with fantastic views of the city.

### Day 2: Florence
**Morning:**
1. **Uffizi Gallery** - Spend your morning immersed in the masterpieces of Renaissance art.

**Lunch:**
2. **Trattoria Mario** - A beloved eatery famous for its tra